In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel('에타 1차 라벨링.xlsx'); df

,Unnamed: 0,date,likes,scrapes,comments,merged,label
0,0,2024-05-13 00:00:00,0,0,"[{'comment': '장게로 가세요', 'likes': 0}]",❤️컴사코 노해선 교수님 족보 정리본 + 과제 코딩 답안 팝니다 ☑️2023학년도 ...,0
1,1,2024-05-13 00:00:00,0,0,"[{'comment': '교바교', 'likes': 0}]",영발 지각 지각하면 점수 많이 깎이나요…?,0
2,2,2024-05-13 00:00:00,0,0,[{'comment': '얼마나 늦었는데? 교바교긴 한데 대체로 하루 이내면 신경 ...,창글 과제 제출? 간단하게 하는 거 늦었는데 A+가능..?ㅠㅠㅠ,1
3,3,2024-05-13 00:00:00,0,0,"[{'comment': 'ㅇㅎㅅ 교수님인가..ㅠ 허탈하겠다 고생했어…..ㅠ', 'l...",와 일찍 일어나서 부랴부랴 컴싸코 다 했는데 이번 주 과제네……………. 10주차는 ...,0
4,4,2024-05-13 00:00:00,0,0,[{'comment': '+지금 보니 다른 곳에 제출했었네요…\n이거 증거로 말씀드...,실험과목 레포트 다 써놓고 바보같이 제출을 안해서\n점수 0점 뜨고나서 알았는데\n...,0
...,...,...,...,...,...,...,...
12406,12406,2019-05-07,0,0,"[{'comment': '재수강입니다.', 'likes': 0}, {'comment...",제가 전학기에 성균논어 F받았는데요 이번 계절에 성균논어 들으면 최초수강취급되나요...,1
12407,12407,2019-05-07,0,0,[{'comment': '과마다 다른 사항이라 미컴 졸업논문 요건은 제가 모릅니다....,사회과학대학 졸업요건에서 미디어커뮤니케이션학과의 경우 논문대신 공인어학성적을 대신 ...,1
12408,12408,2019-05-07,0,0,"[{'comment': '전자공학자가 밤에 애인한테 보내는 문자는???', 'lik...",브로리님 재밌는얘기해주세요,1
12409,12409,2019-05-06,0,0,"[{'comment': '1. 네 2. 네', 'likes': 0}, {'comme...",브로리님\n계절 신청하고 싶은데 전년도 개설과목 찾아보니 전공이 거~의 안열리더라...,1


In [4]:
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel, AutoConfig
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [5]:
from transformers import BertModel

model_checkpoint = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
bert_model = BertModel.from_pretrained(model_checkpoint)
bert_model.to(device)
bert_model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [6]:
# custom dataset class 정의
# 텍스터를 로딩 가능한 데이터로 변경하기 위한 작업.
# tokenizer를 거치면 token_type_ids를 통해 앞 시퀀스인지, 뒤 시퀀스인지 구별 가능

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer # 원본 텍스트를 tokenize할 tokenizer
        self.data = dataframe # 원본 데이터
        self.comment_text = dataframe["merged"]
        self.targets = self.data["label"]
        self.max_len = max_len # tokenizer의 max length argument로 사용

    def __getitem__(self, idx):
        comment_text = str(self.comment_text[idx])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            add_special_tokens = True,
            max_length = self.max_len, # input_max_length
            padding = "max_length", # padding 할 길이
            truncation = True, # truncation 여부
            return_token_type_ids = True, # token_type_ids: 첫 번째 시퀀스(0)인지, 두 번째 시퀀스(1)인지 표시
        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"] #padding을 데이터 처리 하지 않기 위한
        token_type_ids = inputs["token_type_ids"]

        return {
            "ids": torch.tensor(ids, dtype = torch.long),
            "mask": torch.tensor(mask, dtype = torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype = torch.long),
            "targets": torch.tensor(self.targets[idx], dtype = torch.float)
        }

    def __len__(self):
        return len(self.comment_text)

In [7]:
random_seed = 10154315
# train_size = 0.8 #훈련 데이터 사이즈 결정.
# train_dataset = df.sample(frac = train_size, random_state = random_seed).reset_index(drop = True)
# test_dataset = df.drop(train_dataset.index).reset_index(drop = True)

max_len = 300 #토큰화에 사용될 최대 허용 길이

# trainDS = text_D

In [8]:
dataset = TextDataset(df, tokenizer, max_len)
dataloader = DataLoader(dataset, batch_size = 16, shuffle = True) # num_workers param also available
# testDL = DataLoader(testDS, batch_size = 16, shuffle = True)

In [10]:
def generate_embeddings(model, dataloader):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for data in dataloader:
            ids = data['ids'].to(device)
            mask = data['mask'].to(device)
            outputs = model(input_ids=ids, attention_mask=mask)
            # Taking the mean of the token embeddings from the last hidden layer
            embeddings.append(outputs.last_hidden_state.mean(dim=1).cpu().numpy())
    return np.vstack(embeddings)

# Generate and save embeddings
embeddings = generate_embeddings(bert_model, dataloader)
df['embeddings'] = list(embeddings)
df.to_pickle('embeddings.pkl')

In [ ]:
batch = next(iter(dataloader))
batch

In [ ]:
batch['mask'].size()

In [ ]:
trainDS[0]
#id는 실제 데이터, 이중 0의 값은 패딩 값. mask는 패딩 값의 구분

In [ ]:
torch.cuda.is_available()

In [ ]:
!pip install lightgbm

In [13]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the embeddings and labels
embedding_df = pd.read_pickle('embeddings.pkl')
X = np.vstack(embedding_df['embeddings'])
y = df['label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# LightGBM model parameters
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'learning_rate': 0.0001,
    'num_leaves': 31,
    'min_child_samples': 20,
    'colsample_bytree': 0.7,
    'min_child_weight': 0.001,
    'subsample_for_bin': 200000,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1
}

# Train the LightGBM model
gbm = lgb.train(params, train_data, valid_sets=[train_data, test_data], num_boost_round=500)

# Make predictions
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_pred = (y_pred >= 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

[LightGBM] [Info] Number of positive: 4118, number of negative: 5810
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.241504 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 9928, number of used features: 768
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.414786 -> initscore=-0.344213
[LightGBM] [Info] Start training from score -0.344213
Accuracy: 0.5932339911397503


In [ ]:
class Text_Classifier(nn.Module): #분류모델 정의
    def __init__(self, model_checkpoint):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(model_checkpoint) #미리 훈련된  BERT 모델을 불러옴
        self.dropout = nn.Dropout(0.3)
        self.clf = nn.Linear(self.transformer.config.hidden_size, 1) # transformer layer의 hidden size를 최종 class 개수, 2로 압축

    def forward(self, ids, mask, token_type_ids): #순전파 정의
        _, pooled_output = self.transformer(input_ids = ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict = False)
        out = self.dropout(pooled_output)
        out = self.clf(out)

        return out

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Text_Classifier(model_checkpoint)
model = model.to(device)

In [ ]:
learning_rate = 1e-05 #학습률
num_epochs = 20 #에포크 수

#손실함수 정의
def loss_fn(preds, targets): return nn.BCEWithLogitsLoss()(preds, targets)
optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate)

 #훈련과정
def train(epoch):
    model.train()
    total_loss = 0
    for _, batch in enumerate(trainDL):
        ids = batch['ids'].to(device)
        mask = batch['mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        targets = batch['targets'].to(device)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets.view_as(outputs))

        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch}, Loss: {total_loss}")

In [ ]:
for epoch in tqdm(range(num_epochs)):
    train(epoch)

In [ ]:
import numpy as np
from sklearn import metrics
import torch

def validation():
    model.eval()  # 활성화되지 않은 계층을 위해서
    fin_targets = []  # 실제 레이블
    fin_outputs = []  # 출력 레이블
    with torch.no_grad():
        for _, data in enumerate(testDL):  # 배치 단위로 데이터셋 가져옴
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets  # 결과 반환

outputs, targets = validation()
outputs = np.array(outputs) >= 0.5  # 기준 세우기
targets = np.array(targets)  # targets를 numpy 배열로 변환

accuracy = metrics.accuracy_score(targets, outputs)

f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

In [ ]:
def predict(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=True,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    token_type_ids = encoding['token_type_ids'].to(device)

    model.eval()

    with torch.no_grad():
        outputs = model(input_ids, attention_mask, token_type_ids)
        logits = outputs.squeeze()
        prediction = torch.sigmoid(logits).item()

    return 1 if prediction > 0.5 else 0

# 예제 사용법
text = "도전학기랑 계절학기랑 같은 거임?"
label = predict(text)
print(f'Text: {text}\nLabel: {label}')
